In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
from IPython.display import HTML
import pickle
import time
# from google.colab import files

In [2]:
import numpy as np
from pathlib import Path
from torchvision import transforms
import torch
from torch import nn
import torchvision.utils as vutils

In [3]:
from utils import visual_data, load_cifar10
from networks import Generator, Discriminator, weights_init, DiscriminatorMiniBatchDiscrimination, DCGAN
from trainer import  update_params
from losses import D_loss, G_featMatch_loss, G_loss

In [4]:
!nvidia-smi

Wed Apr 15 21:55:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 960     Off  | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P0    39W / 128W |    576MiB /  4040MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
# Setting up constants
# device
ngpu = 1

# single image
imageSize = 64
imageMean = (0.4923172 , 0.48307145, 0.4474483)
imageStd = (0.24041407, 0.23696952, 0.25565723)

# data loader
numWorkers = 3
batchSize = 16

# Network Arch
nc = 3 # Number of channels
nz = 100 # Latent vector
ngf = 64 # relates to the depth of feature maps carried through the generator
ndf = 64 # sets the depth of feature maps propagated through the discriminator

# Training
num_epochs = 5

# Adam Optimizer
lr = .0002
beta1 = .5

# convention of the labeling for the real and the fake datasets
## one-sided label smoothing
real_label = .9
fake_label = 0

# label smoothing
## insdead of real label=.9 give uniform between .8, 1
## insdead of fake label=0 give uniform between 0, .2
label_smoothing = True

# flip labels, with probability pFlip, flip the labels passed to the discriminator
pFlip = 0.05

# if to use last two layers' features for feature matching
double_layer = True

# whether to do minibatch normalization
miniBatchDiscrimination = True

In [6]:
# folder to store/load data
dataFolder = Path("./data")
# Decide which device to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [7]:
torch.cuda.is_available()

True

In [8]:
cifarFolder = dataFolder/"CIFAR10"
# prepare data for loading
tsfms = transforms.Compose([
    transforms.Resize(imageSize), 
    transforms.ToTensor(),
    transforms.Normalize(imageMean, imageStd)
])
trainLoader, test_loader = load_cifar10(cifarFolder, tsfms, batchSize, numWorkers)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# init D and G network
gen = Generator(ngpu, nz, ngf, nc).to(device)
gen.apply(weights_init);

if not miniBatchDiscrimination:
    disc = Discriminator(ngpu, nc, ndf).to(device)
else: 
    disc = DiscriminatorMiniBatchDiscrimination(ngpu, nc, ndf).to(device)

disc.apply(weights_init);

In [10]:
dcgan = DCGAN(gen, disc, device, 
              real_label, fake_label, pFlip, label_smoothing, double_layer)

In [11]:
# setup optmization 
optimizerD = torch.optim.Adam(dcgan.discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
# optimizerD = torch.optim.SGD(disc.parameters(), lr=0.1, momentum=0.9)
optimizerG = torch.optim.Adam(dcgan.generator.parameters(), lr=lr, betas=(beta1, 0.999))

In [12]:
# fixed noise z for viusalization of the progress of the training
fixed_noise = torch.randn(batchSize, dcgan.generator.nz, 1, 1, device=device)

# training loop 
# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
g=0
print("Starting Training Loop...")

Starting Training Loop...


In [13]:
num_epochs = 30

In [15]:
from gan import trainer_GAN
trainer = trainer_GAN(dcgan, trainLoader, optimizerD, optimizerG)
trainer.train()

[0000/3125]  Loss_D: 0.664  Loss_G: 1.011 t= 0.095
[0050/3125]  Loss_D: 0.711  Loss_G: 0.995 t= 0.600
[0100/3125]  Loss_D: 0.689  Loss_G: 1.018 t= 0.612
[0150/3125]  Loss_D: 0.795  Loss_G: 0.918 t= 0.546
[0200/3125]  Loss_D: 0.697  Loss_G: 0.979 t= 0.615
[0250/3125]  Loss_D: 1.787  Loss_G: 0.000 t= 0.603
[0300/3125]  Loss_D: 1.897  Loss_G: 0.000 t= 0.584
[0350/3125]  Loss_D: 0.734  Loss_G: 0.896 t= 0.566
[0400/3125]  Loss_D: 0.715  Loss_G: 0.995 t= 0.603
[0450/3125]  Loss_D: 0.621  Loss_G: 0.977 t= 0.525
[0500/3125]  Loss_D: 0.619  Loss_G: 0.999 t= 0.584
[0550/3125]  Loss_D: 0.718  Loss_G: 0.965 t= 0.604
[0600/3125]  Loss_D: 0.682  Loss_G: 0.917 t= 0.596
[0650/3125]  Loss_D: 0.727  Loss_G: 0.903 t= 0.607
[0700/3125]  Loss_D: 0.597  Loss_G: 0.971 t= 0.625
[0750/3125]  Loss_D: 1.502  Loss_G: 0.000 t= 0.544
[0800/3125]  Loss_D: 0.720  Loss_G: 0.883 t= 0.599
[0850/3125]  Loss_D: 0.598  Loss_G: 0.951 t= 0.615
[0900/3125]  Loss_D: 0.717  Loss_G: 0.875 t= 0.691
[0950/3125]  Loss_D: 2.418  Los

Process Process-4:
Process Process-5:
Traceback (most recent call last):
Process Process-6:
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _wo

KeyboardInterrupt: 

  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/selectors.py", line 376, in select
    fd

In [ ]:
g=0
# For each epoch
current = time.time()
# for each epoch
for epoch in range(num_epochs):
    # For each batch
    for i, data in enumerate(trainLoader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################   ` 

        # real batch and fake batch
        real_batch = data[0].to(device)
        fake_batch = dcgan.fake_batch(real_batch.size(0))
        
        errD = dcgan.discriminator_loss(real_batch, fake_batch)
        
        update_params(optimizerD, errD)
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################

        if errD.item() < .8 or g==0:
            g += 1
            fake_batch = dcgan.fake_batch(real_batch.size(0))
            
            errG = dcgan.generator_loss(real_batch, fake_batch)

            update_params(optimizerG, errG)

        ###########################
        # (3) save and print progress
        ###########################
#         # for progress prints
#         D_x = output_real.mean().item()
#         D_G_z1 = output_fake.mean().item()
#         D_G_z2 = output.mean().item()
        
#         # Output training stats
#         if i % 50 == 0:
#             print('[{:0>2}/{:0>2}][{:0>4}/{:0>4}]  Loss_D: {:.3f}  Loss_G: {:.3f}  D(x): {:.3f}  D(G(z)): {:.3f} / {:.3f}  t={:6.3f}  iterG={}/50'.format(
#                 epoch, num_epochs, i, len(trainLoader),errD.item(), errG.item(), D_x, D_G_z1, D_G_z2, time.time()-current, g, prec=3))
#             g = 0
#             current = time.time()

        if i % 50 == 0:
            print('[{:0>4}/{:0>4}]  Loss_D: {:.3f}  Loss_G: {:.3f} t={:6.3f}'.format(
                    i, len(trainLoader), errD, errG, time.time()-current, prec=3))
            g = 0
            current = time.time()

        # Save Losses for plotting later
#         G_losses.append(errG.item())
#         D_losses.append(errD.item())

#         # Check how the generator is doing by saving G's output on fixed_noise
#         if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(trainLoader)-1)):
#             clear_output()
#             with torch.no_grad():
#                 fake = gen(fixed_noise).detach().cpu()
#             img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
g=0
# For each epoch
current = time.time()
# for each epoch
for epoch in range(num_epochs):
    # For each batch
    for i, data in enumerate(trainLoader, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################   ` 

        # real batch and fake batch
        real_batch = data[0].to(device)
        fake_batch = dcgan.fake_batch(real_batch.size(0))
        
        errD = dcgan.discriminator_loss(real_batch, fake_batch)
        
        update_params(optimizerD, errD)
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################

        if errD.item() < .8 or g==0:
            g += 1
            fake_batch = dcgan.fake_batch(real_batch.size(0))
            
            errG = dcgan.generator_loss(real_batch, fake_batch)
            
            update_params(optimizerG, errG)

        ###########################
        # (3) save and print progress
        ###########################
#         # for progress prints
#         D_x = output_real.mean().item()
#         D_G_z1 = output_fake.mean().item()
#         D_G_z2 = output.mean().item()
        
#         # Output training stats
#         if i % 50 == 0:
#             print('[{:0>2}/{:0>2}][{:0>4}/{:0>4}]  Loss_D: {:.3f}  Loss_G: {:.3f}  D(x): {:.3f}  D(G(z)): {:.3f} / {:.3f}  t={:6.3f}  iterG={}/50'.format(
#                 epoch, num_epochs, i, len(trainLoader),errD.item(), errG.item(), D_x, D_G_z1, D_G_z2, time.time()-current, g, prec=3))
#             g = 0
#             current = time.time()

        if i % 50 == 0:
            print('[{:0>4}/{:0>4}]  Loss_D: {:.3f}  Loss_G: {:.3f} t={:6.3f}'.format(
                    i, len(trainLoader), errD, errG, time.time()-current, prec=3))
            g = 0
            current = time.time()

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

#         # Check how the generator is doing by saving G's output on fixed_noise
#         if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(trainLoader)-1)):
#             clear_output()
#             with torch.no_grad():
#                 fake = gen(fixed_noise).detach().cpu()
#             img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
len(G_losses)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D", alpha=.5)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
trainIter = iter(trainLoader)

In [ ]:
real_batch = next(trainIter)

noise = torch.randn(64, nz, 1, 1, device=device)
fake = gen(noise).detach().cpu()
fake = vutils.make_grid(fake, padding=2, normalize=True)

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(fake,(1,2,0)))
plt.show()

In [ ]:
# print(optimizerD, optimizerG, trainLoader.batch_size, ngf, ndf, real_label)

In [ ]:
# genParams = gen.state_dict()
# discParams = disc.state_dict()

In [ ]:
def dumpAndDL(variable, fileName):
    with open(fileName, 'wb') as f:
        pickle.dump(variable,
                    f)
        # files.download(fileName)
        
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
# dumpAndDL(
#     {'genParams': genParams,
#      'discParams': discParams,
#     },
#     'bestParams.p'
# )

In [ ]:
dumpAndDL(
    {'G_losses': G_losses,
     'D_losses':D_losses,
     },
     'losses.p'
       )

In [ ]:
n=len(img_list)

for idx, chunk in enumerate(chunks(img_list, n)):
    fileName = 'img_list_{}.p'.format(idx)
    dumpAndDL(
        chunk,
        fileName
    )

In [ ]:
# # load params to networks
# fileName = 'bestParams.p'
# netParams = None
# with open(fileName, 'rb') as f:
#      netParams = pickle.load(f)
# gen.load_state_dict(netParams['genParams'])
# disc.load_state_dict(netParams['discParams'])

In [ ]:
# # load params to networks
# fileNames = ['img_list_0.p', 'img_list_1.p', 'img_list_2.p', 'img_list_3.p']

# img_list = []
# for fileName in fileNames:
#     with open(fileName, 'rb') as f:
#         img_list += pickle.load(f)['img_list']


In [ ]:
# len(img_list)

In [ ]:
# #%%capture animation
# fig = plt.figure(figsize=(10, 10))
# plt.axis("off")
# ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list[:100]]
# ani = animation.ArtistAnimation(fig, ims, interval=100, repeat_delay=1000, blit=True)

# HTML(ani.to_jshtml())

# # writer = animation.writers['ffmpeg']
# # ani.save('im.mp4', writer=writer, dpi=100)